In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# ---------------------------------------
# 1. Split the data and move it to device
# ---------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train, y_test = y_train.squeeze(), y_test.squeeze()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Convert to PyTorch tensors and move to device
x_train_tensor = torch.as_tensor(X_train).float().to(device)
x_test_tensor = torch.as_tensor(X_test).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device).unsqueeze(1)  # [N, 1]
y_test_tensor = torch.as_tensor(y_test).float().to(device).unsqueeze(1)    # [N, 1]

# Reshape to [batch_size, seq_len, input_size]
x_train_tensor = x_train_tensor.unsqueeze(-1)  # [N, L, 1]
x_test_tensor = x_test_tensor.unsqueeze(-1)

# ---------------------------------------
# 2. Define the LSTM model
# ---------------------------------------
class GWClassifierLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, num_layers=3):
        super(GWClassifierLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 16),
            nn.LeakyReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out, _ = self.lstm(x)         # out: [batch, seq_len, hidden_size]
        out = out[:, -1, :]           # take last time step
        out = self.fc(out)            # fully connected classifier
        return out

model = GWClassifierLSTM().to(device)

# ---------------------------------------
# 3. Define loss and optimizer with momentum
# ---------------------------------------
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# ---------------------------------------
# 4. Train the model
# ---------------------------------------
epochs = 100
loss_vals = []

for epoch in range(epochs):
    model.train()
    y_pred = model(x_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    loss_vals.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d} | Loss: {loss.item():.4f}")

# Optional: plot loss
plt.plot(loss_vals)
plt.xlabel("Epoch")
plt.ylabel("Binary Cross-Entropy Loss")
plt.title("Training Loss Curve (LSTM)")
plt.grid(True)
plt.tight_layout()
plt.show()
